# 01. Traffic Sign Classification - 모델 학습

이 노트북은 Hugging Face의 사전학습된 CNN 모델을 사용하여 교통표지판을 분류하는 모델을 학습합니다.

## 학습 과정
1. 환경 설정 및 라이브러리 설치
2. Google Drive 마운트 및 데이터 로드
3. 로컬에 저장된 데이터셋 불러오기
4. 사전학습된 모델 로드 및 Fine-tuning
5. 학습 결과 저장

## 1. 환경 설정

In [ ]:
# Hugging Face transformers 및 관련 라이브러리 설치
!pip install transformers pillow torch torchvision accelerate -q

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset as TorchDataset
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pathlib import Path

print("✅ 라이브러리 로드 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 작업 디렉토리 설정

**중요**: 아래 경로를 본인의 Google Drive 구조에 맞게 수정하세요.

In [ ]:
# Google Drive 내 작업 디렉토리 경로 설정
WORK_DIR = '/content/drive/MyDrive/2026_AI_Advanced_Study-main/3차시/02_Traffic_Sign_Classification'

# 작업 디렉토리로 이동
os.chdir(WORK_DIR)
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 데이터 경로 확인
DATA_DIR = Path('data/images')
if DATA_DIR.exists():
    print(f"✅ 데이터 디렉토리 발견: {DATA_DIR}")
else:
    print(f"❌ 데이터 디렉토리를 찾을 수 없습니다: {DATA_DIR}")

## 4. 커스텀 데이터셋 클래스 정의

In [ ]:
class TrafficSignDataset(TorchDataset):
    """로컬 이미지 파일을 로드하는 커스텀 데이터셋"""
    
    def __init__(self, data_dir, processor=None):
        self.data_dir = Path(data_dir)
        self.processor = processor
        self.samples = []
        
        # 클래스별 이미지 수집
        for class_idx in range(5):
            class_dir = self.data_dir / f'class_{class_idx}'
            if class_dir.exists():
                for img_path in sorted(class_dir.glob('*.jpg')):
                    self.samples.append((img_path, class_idx))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        
        # 전처리 적용
        if self.processor is not None:
            inputs = self.processor(images=image, return_tensors='pt')
            # 배치 차원 제거
            pixel_values = inputs['pixel_values'].squeeze(0)
            return {'pixel_values': pixel_values, 'label': label}
        else:
            return {'image': image, 'label': label}

print("✅ 커스텀 데이터셋 클래스 정의 완료")

## 5. 이미지 프로세서 로드

In [ ]:
# 사전학습된 모델의 Image Processor 로드
MODEL_NAME = "google/mobilenet_v2_1.0_224"  # 간단한 CNN 모델

print(f"📦 이미지 프로세서 로드: {MODEL_NAME}")
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)

print("✅ 이미지 프로세서 로드 완료!")

## 6. 데이터셋 로드

In [ ]:
print("📥 로컬 데이터셋 로드 중...\n")

# 데이터셋 생성
train_dataset = TrafficSignDataset('data/images/train', processor=processor)
val_dataset = TrafficSignDataset('data/images/val', processor=processor)
test_dataset = TrafficSignDataset('data/images/test', processor=processor)

print(f"✅ 데이터셋 로드 완료!")
print(f"  - Train: {len(train_dataset)}장")
print(f"  - Val: {len(val_dataset)}장")
print(f"  - Test: {len(test_dataset)}장")
print(f"\n📋 클래스 정보: 5개 교통 표지판 클래스 (Class 0 ~ Class 4)")

## 7. 모델 로드 및 설정

Hugging Face의 사전학습된 MobileNetV2 모델을 로드하고, 5개 클래스 분류를 위해 마지막 레이어를 조정합니다.

In [ ]:
# 사전학습된 모델 로드
print(f"🤖 모델 로드: {MODEL_NAME}")

model = AutoModelForImageClassification.from_pretrained(
    MODEL_NAME,
    num_labels=5,  # 5개 클래스
    id2label={i: f"Class {i}" for i in range(5)},
    label2id={f"Class {i}": i for i in range(5)},
    ignore_mismatched_sizes=True
)

print(f"✅ 모델 로드 완료!")
print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

## 8. 학습 설정

In [ ]:
# 평가 메트릭 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# 학습 설정
training_args = TrainingArguments(
    output_dir='runs/classification',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_dir='runs/classification/logs',
    logging_strategy='epoch',  # Epoch 단위로 로그 기록
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='none'
)

print("=== 학습 설정 ===")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Batch Size: {training_args.per_device_train_batch_size}")
print(f"Learning Rate: {training_args.learning_rate}")
print(f"결과 저장 경로: {training_args.output_dir}")

## 9. 모델 학습

In [ ]:
# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("🚀 학습 시작...\n")
train_result = trainer.train()

print("\n✅ 학습 완료!")
print(f"학습된 모델 저장 위치: runs/classification")

## 10. 학습 결과 저장

In [ ]:
# 최종 모델 저장
trainer.save_model('runs/classification/final_model')
processor.save_pretrained('runs/classification/final_model')

print("✅ 모델 및 설정 저장 완료!")
print(f"\n다음 단계:")
print("  1. 02_test.ipynb에서 모델 성능 평가")
print("  2. 03_visualize.ipynb에서 예측 결과 시각화")

## 11. 학습 곡선 시각화

In [ ]:
import matplotlib.pyplot as plt

# 학습 로그 로드
log_history = trainer.state.log_history

# Epoch 단위로 기록된 로그만 추출
train_logs = [log for log in log_history if 'loss' in log and 'epoch' in log and 'eval_loss' not in log]
eval_logs = [log for log in log_history if 'eval_loss' in log]

if len(eval_logs) > 0:
    # 그래프 그리기
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss 그래프 (Epoch 기준)
    train_loss = [log['loss'] for log in train_logs]
    eval_loss = [log['eval_loss'] for log in eval_logs]
    epochs = list(range(1, len(eval_loss) + 1))
    
    axes[0].plot(epochs, train_loss, label='Train Loss', marker='o')
    axes[0].plot(epochs, eval_loss, label='Val Loss', marker='s')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Accuracy 그래프 (Epoch 기준)
    eval_acc = [log['eval_accuracy'] for log in eval_logs]
    axes[1].plot(epochs, eval_acc, label='Validation Accuracy', marker='s', color='green')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig('runs/classification/training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ 학습 곡선 저장: runs/classification/training_curves.png")
else:
    print("⚠️ 평가 로그가 없습니다.")